# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robersta, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów.
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["jet.png", "kw.png", "moon.png", "lenaSzum.png", "lena.png", "plansza.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)



In [ ]:
def show_img_with_hist(img, title):
  _, ax = plt.subplots(1, 2,figsize=(20, 5), dpi=200)
  ax[0].imshow(img, cmap="gray")
  ax[0].set_title(title)
  ax[1].plot(cv2.calcHist([img], [0], None, [256], [0, 256]))
  ax[1].set_title("histogram")

In [ ]:
board = cv2.imread('plansza.png', cv2.IMREAD_GRAYSCALE)
board_2D_filter = cv2.filter2D (board, -1, np.ones((3,3), dtype=np.float32)/9)
board_2D_filter_scipy = signal.convolve2d(board, np.ones((3,3), dtype=np.float32)/9, mode='same')
show_img_with_hist(board, "board")
show_img_with_hist(board_2D_filter, "board_2D_filter")
show_img_with_hist(board_2D_filter_scipy, "board_2D_filter_scipy")

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35.
Wykorzystaj polecenie `plt.subplot`.
Przeanalizuj wpływ rozmiaru maski na wynik.

In [ ]:
filter_list = [signal.convolve2d(board, np.ones((i, i)) / (i * i), mode='same') for i in [3,5,9,15,35]]
_, ax = plt.subplots(1, 5,figsize=(20, 5), dpi=200)
for i in range(len(filter_list)):
  ax[i].imshow(filter_list[i], cmap="gray")
  ax[i].set_title("filter size: " + str([3,5,9,15,35][i]))

In [ ]:
lena = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
filter_list = [signal.convolve2d(lena, np.ones((i, i)) / (i * i), mode='same') for i in [3,5,9,15,35]]
_, ax = plt.subplots(1, 5,figsize=(20, 5), dpi=200)
for i in range(len(filter_list)):
  ax[i].imshow(filter_list[i], cmap="gray")
  ax[i].set_title("filter size: " + str([3,5,9,15,35][i]))

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
lena = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
lena_2D_filter = cv2.filter2D (lena, -1, np.ones((3,3), dtype=np.float32)/9)
lena_2D_filter_scipy = signal.convolve2d(lena, np.ones((3,3), dtype=np.float32)/9, mode='same')
show_img_with_hist(lena, "lena")
show_img_with_hist(lena_2D_filter, "lena_2D_filter")
show_img_with_hist(lena_2D_filter_scipy, "lena_2D_filter_scipy")
show_img_with_hist(lena - lena_2D_filter_scipy, "lena_roznica")

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości.
Częściowo można go zniwelować poprzez odpowiedni dobór maski.
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`.
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
M = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
M = M / sum(M)
lena_2D_filter_M = signal.convolve2d(lena, M, mode='same')
plt.imshow(lena_2D_filter_M, cmap="gray",)

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa.
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr ``odchylenie standardowe'' wpływa na ``kształt'' filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum()


def mesh(fun, size):
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')


    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun

    ax.plot_surface(X, Y, Z)

    plt.show()

In [ ]:
lena_blur_Gaussian = cv2.GaussianBlur(lena, (5, 5), 0)
show_img_with_hist(lena_blur_Gaussian, "lena_blur_Gaussian")
lena_blur_Gaussian_scipy = signal.convolve2d(lena, fgaussian(5, 0.5), mode='same')
_, ax = plt.subplots(1, 3,figsize=(20, 5), dpi=200)
ax[0].imshow(lena_blur_Gaussian_scipy, cmap="gray")
ax[0].set_title("lena_blur_Gaussian_scipy")
ax[1].imshow(signal.convolve2d(lena, fgaussian(5, 2), mode='same'), cmap="gray")
ax[1].set_title("lena_blur_Gaussian_scipy sigma=2")
ax[2].imshow(lena_blur_Gaussian - lena_blur_Gaussian_scipy, cmap="gray")
ax[2].set_title("lena_blur_Gaussian - lena_blur_Gaussian_scipy")


In [ ]:
mesh(fgaussian(5, 0.5), 5)

### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję ``cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lena_szum = cv2.imread('lenaSzum.png', cv2.IMREAD_GRAYSCALE)
lena_szum_2D_filter = signal.convolve2d(lena_szum, np.ones((3,3), dtype=np.float32)/9, mode='same')
show_img_with_hist(lena_szum, "lena_szum")
show_img_with_hist(lena_szum_2D_filter, "lena_szum_2D_filter")
show_img_with_hist((lena_szum - lena_szum_2D_filter), "lena_szum - lena_szum_2D_filter ")

lena_szum_medfilt2d = signal.medfilt2d(lena_szum, kernel_size=3)
show_img_with_hist(lena_szum_medfilt2d, "lena_szum_medfilt2d")
show_img_with_hist((lena_szum - lena_szum_medfilt2d), "lena_szum - lena_szum_medfilt2d")

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
lena_2D_filter = signal.convolve2d(lena, np.ones((3,3), dtype=np.float32)/9, mode='same')
show_img_with_hist(lena, "lena")
show_img_with_hist(lena_2D_filter, "lena_2D_filter")
show_img_with_hist((lena - lena_2D_filter), "lena - lena_2D_filter")

lena_2D_medfilt2d = signal.medfilt2d(lena, kernel_size=3)
show_img_with_hist(lena_2D_medfilt2d, "lena_2D_medfilt2d")
show_img_with_hist((lena - lena_2D_medfilt2d), "lena - lena_2D_medfilt2d")


3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt ``x'',``+''), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z ``x'' i mediana z ``+'',
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
lena_loop = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)

for i in range(8):
    before = lena_loop
    lena_loop = signal.medfilt2d(lena_loop, kernel_size=5)
    _, ax = plt.subplots(1, 3,figsize=(20, 5), dpi=200)
    ax[0].imshow(before, cmap="gray")
    ax[0].set_title("image before filter in loop"+ str(i))
    ax[1].imshow(lena_loop, cmap="gray")
    ax[1].set_title("image after filter in loop" + str(i))
    ax[2].imshow(before - lena_loop, cmap="gray")
    ax[2].set_title("image difference in loop" + str(i))

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M =
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez 9.
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowanych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodatnie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje.
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach.

In [ ]:
moon = cv2.imread("moon.png", cv2.IMREAD_GRAYSCALE).astype(np.float32)
lepsian_mask = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]]) / 9
moon_lepsian_mask = signal.convolve2d(moon, lepsian_mask, mode="same")
moon_lepsian_mask_normalized = cv2.normalize(moon_lepsian_mask, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
moon_lepsian_mask_abs = np.abs(moon_lepsian_mask)
_ , ax = plt.subplots(1, 5,figsize=(20, 5), dpi=200)
ax[0].imshow(moon, cmap="gray")
ax[0].set_title("moon")
ax[1].imshow(moon_lepsian_mask, cmap="gray")
ax[1].set_title("moon_lepsian_mask")
ax[2].imshow(moon - moon_lepsian_mask, cmap="gray")
ax[2].set_title("moon - moon_lepsian_mask")
ax[3].imshow(moon_lepsian_mask_normalized, cmap="gray")
ax[3].set_title("moon_lepsian_mask_normalized")
ax[4].imshow(moon_lepsian_mask_abs, cmap="gray")
ax[4].set_title("moon_lepsian_mask_abs")





7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
_, ax = plt.subplots(1, 3,figsize=(20, 5), dpi=200)
ax[0].imshow(moon, cmap="gray")
ax[0].set_title("moon")
ax[1].imshow(moon + moon_lepsian_mask_abs, cmap="gray")
ax[1].set_title("moon + moon_lepsian_mask_abs")
ax[2].imshow(abs(moon - moon_lepsian_mask_abs), cmap="gray")
ax[2].set_title("moon - moon_lepsian_mask_abs")

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie ``prostopadłe''):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela.

In [ ]:
kw = cv2.imread("kw.png", cv2.IMREAD_GRAYSCALE).astype(np.float32)
R1 = np.array([[0, 0, 0], [-1, 1, 0], [0, 0, 0]])
R2 = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
P1 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
P2 = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

In [ ]:
def edge_detection(img, mask, title):
    result = signal.convolve2d(img, mask, mode="same")
    result_normalized = cv2.normalize(result, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    result_abs = np.abs(result)
    _, ax = plt.subplots(1, 6,figsize=(20, 5), dpi=200)
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title(title)
    ax[1].imshow(result, cmap="gray")
    ax[1].set_title("result")
    ax[2].imshow(result_normalized, cmap="gray")
    ax[2].set_title("result_normalized")
    ax[3].imshow(result_abs, cmap="gray")
    ax[3].set_title("result_abs")
    ax[4].imshow(img + result_abs, cmap="gray")
    ax[4].set_title("img + result_abs")
    ax[5].imshow(abs(img - result_abs), cmap="gray")
    ax[5].set_title("img - result_abs")


In [ ]:
edge_detection(kw, R1, "kw R1")
edge_detection(kw, R2, "kw R2")
edge_detection(kw, P1, "kw P1")
edge_detection(kw, P2, "kw P2")
edge_detection(kw, S1, "kw S1")
edge_detection(kw, S2, "kw S2")

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
def edge_detection_combined(img, mask1, mask2, title):
    result1 = signal.convolve2d(img, mask1, mode="same")
    result2 = signal.convolve2d(img, mask2, mode="same")
    result = np.sqrt(result1**2 + result2**2)
    result_normalized = cv2.normalize(result, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    result_abs = np.abs(result)
    _, ax = plt.subplots(1, 6,figsize=(20, 5), dpi=200)
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title(title)
    ax[1].imshow(result, cmap="gray")
    ax[1].set_title("result")
    ax[2].imshow(result_normalized, cmap="gray")
    ax[2].set_title("result_normalized")
    ax[3].imshow(result_abs, cmap="gray")
    ax[3].set_title("result_abs")
    ax[4].imshow(img + result_abs, cmap="gray")
    ax[4].set_title("img + result_abs")
    ax[5].imshow(abs(img - result_abs), cmap="gray")
    ax[5].set_title("img - result_abs")
    


In [ ]:
edge_detection_combined(kw, S1, S2, "kw S1 S2")

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia).
Zaimplementuj tę wersję.

In [ ]:
def edge_detection_combined_no_sqrt(img, mask1, mask2, title):
    result1 = signal.convolve2d(img, mask1, mode="same")
    result2 = signal.convolve2d(img, mask2, mode="same")
    result = np.abs(result1) + np.abs(result2)
    result_normalized = cv2.normalize(result, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    result_abs = np.abs(result)
    _, ax = plt.subplots(1, 6,figsize=(20, 5), dpi=200)
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title(title)
    ax[1].imshow(result, cmap="gray")
    ax[1].set_title("result")
    ax[2].imshow(result_normalized, cmap="gray")
    ax[2].set_title("result_normalized")
    ax[3].imshow(result_abs, cmap="gray")
    ax[3].set_title("result_abs")
    ax[4].imshow(img + result_abs, cmap="gray")
    ax[4].set_title("img + result_abs")
    ax[5].imshow(abs(img - result_abs), cmap="gray")
    ax[5].set_title("img - result_abs")

In [ ]:
edge_detection_combined_no_sqrt(kw, S1, S2, "kw S1 S2 no sqrt")

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:

jet = cv2.imread("jet.png", cv2.IMREAD_GRAYSCALE).astype(np.float32)
edge_detection(jet, S1, "jet S1")
edge_detection(jet, S2, "jet S2")
edge_detection_combined(jet, S1, S2, "jet S1 S2")
edge_detection_combined_no_sqrt(jet, S1, S2, "jet S1 S2 no sqrt")